In [1]:
!which python

/Users/ahmadabdullahtariq/Documents/Projects/Thesis/.venv/bin/python


In [2]:
import pandas as pd
import numpy as np 
import plotly.graph_objects as go

In [25]:
df = pd.read_pickle('../dataset/preprocessed/284_imputed.pkl')

In [26]:
print(df.isna().sum())
print(df.dtypes)

TargetDate      0
Definitive      0
Rooms           0
SnapshotDate    0
LAR             0
dtype: int64
TargetDate      datetime64[ns]
Definitive             float64
Rooms                  float64
SnapshotDate    datetime64[ns]
LAR                    float64
dtype: object


# Pick Up Model 
**Idea:** Classicial model also known as advance booking models. They use development of bookings over time for stay date to predict final date. The key concept here is how much the value has increased over time, and this gives us the trend for increments over time for prediction.
<br><br>
**Types**: There are two types of pick-up models.<br>
1. Additive <br>
2. Multiplicative <br>

## Additive
\begin{gather*} 
ROH_{0,t} = ROH_{today} + P_{i,t}\\
P_{i,t} = \frac{\sum_{i=1}^{n}{True_t-ROH_{0,t}}}{n}
\end{gather*}

In [27]:
df['LeadTime'] = (df['TargetDate']-df['SnapshotDate']).dt.days
# df['DOW'] = df.TargetDate.dt.dayofweek
df = df.drop(columns=['LAR','SnapshotDate'])
# just rearrange columns 
df = df[['TargetDate','LeadTime','Rooms','Definitive']]

In [28]:
df.head()

,TargetDate,LeadTime,Rooms,Definitive
0,2018-01-01,88,290.0,58.0
1,2018-01-01,87,290.0,58.0
2,2018-01-01,85,290.0,58.0
3,2018-01-01,81,290.0,63.0
4,2018-01-01,80,290.0,62.0


In [29]:
df = df.pivot(index='TargetDate',columns='LeadTime',values='Definitive')
df = df.ffill(axis=1).bfill(axis=1)
# Consider only 2018 and 2019
df = df.iloc[:719]

In [30]:
df.head()

LeadTime,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
TargetDate,,,,,,,,,,,,,,,,,,,,,
2018-01-01,209.0,206.0,206.0,198.0,204.0,198.0,196.0,199.0,197.0,189.0,...,62.0,63.0,63.0,63.0,63.0,58.0,58.0,58.0,58.0,58.0
2018-01-02,223.0,214.0,209.0,209.0,201.0,203.0,188.0,183.0,184.0,185.0,...,52.0,50.0,51.0,51.0,51.0,51.0,49.0,49.0,48.0,48.0
2018-01-03,238.0,218.0,212.0,207.0,207.0,194.0,195.0,174.0,168.0,169.0,...,40.0,40.0,39.0,39.0,39.0,39.0,39.0,36.0,36.0,36.0
2018-01-04,234.0,222.0,211.0,208.0,203.0,203.0,189.0,186.0,161.0,152.0,...,35.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,32.0,32.0
2018-01-05,261.0,238.0,223.0,212.0,210.0,207.0,207.0,194.0,185.0,158.0,...,29.0,29.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,26.0


In [43]:
dx = df.copy()
result = 0
for col in range(9,89):
    result = result + (dx[0]-dx[col])


In [9]:
dx = df.copy()
for col in range(0,89):
    df[col] = dx[col]-dx[col+1]

In [10]:
df.head()

LeadTime,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
TargetDate,,,,,,,,,,,,,,,,,,,,,
2018-01-01,3.0,0.0,8.0,-6.0,6.0,2.0,-3.0,2.0,8.0,-1.0,...,-1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,58.0
2018-01-02,9.0,5.0,0.0,8.0,-2.0,15.0,5.0,-1.0,-1.0,8.0,...,2.0,-1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,48.0
2018-01-03,20.0,6.0,5.0,0.0,13.0,-1.0,21.0,6.0,-1.0,-2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,36.0
2018-01-04,12.0,11.0,3.0,5.0,0.0,14.0,3.0,25.0,9.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,32.0
2018-01-05,23.0,15.0,11.0,2.0,3.0,0.0,13.0,9.0,27.0,12.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,26.0


In [11]:
print("Target Date 2018-01-02 final occupancy:",np.sum(df.iloc[1].values))

Target Date 2018-01-02 final occupancy: 223.0


In [17]:
train = df.iloc[:575]
test = df.iloc[575:]

In [19]:
# sum all columns and average
p = train.sum(axis = 0)/575

In [24]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=p.index, y=p,mode='lines',name='Price'))
fig.show()   

## Multiplicative

In [39]:
from operator import mul
from functools import reduce

In [36]:
df.head()

LeadTime,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
TargetDate,,,,,,,,,,,,,,,,,,,,,
2018-01-01,209.0,206.0,206.0,198.0,204.0,198.0,196.0,199.0,197.0,189.0,...,62.0,63.0,63.0,63.0,63.0,58.0,58.0,58.0,58.0,58.0
2018-01-02,223.0,214.0,209.0,209.0,201.0,203.0,188.0,183.0,184.0,185.0,...,52.0,50.0,51.0,51.0,51.0,51.0,49.0,49.0,48.0,48.0
2018-01-03,238.0,218.0,212.0,207.0,207.0,194.0,195.0,174.0,168.0,169.0,...,40.0,40.0,39.0,39.0,39.0,39.0,39.0,36.0,36.0,36.0
2018-01-04,234.0,222.0,211.0,208.0,203.0,203.0,189.0,186.0,161.0,152.0,...,35.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,32.0,32.0
2018-01-05,261.0,238.0,223.0,212.0,210.0,207.0,207.0,194.0,185.0,158.0,...,29.0,29.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,26.0


In [37]:
dx = df.copy()
for col in range(0,89):
    df[col] = dx[col]/dx[col+1]

In [41]:
reduce(mul, df.iloc[0].values)

209.0000000000001